# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='../Reacher_Windows_x86_64_2/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [10]:
from ddpg_agent import Agent

agent = Agent(state_size=33, action_size=4, random_seed=10)

In [11]:
from ddpg_agent import device

In [12]:
device

device(type='cuda', index=0)

In [13]:
from collections import deque
import torch

In [19]:
def ddpg(n_episodes=1000, max_t=4000):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        states = env_info.vector_observations           # get the current state of all agents
        agent.reset()
        score = np.zeros(20) 
        for t in range(max_t):
            actions=agent.act(states)
            env_info = env.step(actions)[brain_name]        # send the action to the environment
            next_states = env_info.vector_observations   # get the next state
            rewards = env_info.rewards                  # get the reward
            dones = env_info.local_done                  # see if episode has finished
            for i in range(20):
                agent.step(states[i], actions[i], rewards[i], next_states[i], dones[i],t)
            
            score+= rewards
            states = next_states    
            if np.any(dones):
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque))) 
        if (np.mean(scores_deque)> max_score):
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_1.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_1.pth')
            max_score=np.mean(scores_deque)
        if (max_score > 30):
            print('\rEnvirnoment Solved in Episode {}\t and Average Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            break
                          
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 61	Average Score: 0.05

KeyboardInterrupt: 

In [14]:
def train_ddpg(num_agents=20, n_episodes=1000, max_t=1000):

    episode_scores = []                                    # list containing scores from each episode
    scores_window = deque(maxlen=100)

    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations              # get the current state (for each agent)
        agent.reset()
        score = np.zeros(num_agents)                       # initialize the score (for each agent)
        for t in range(max_t):
            actions = agent.act(states)                    # select an action (for each agent):
            # actions = np.clip(actions, -1, 1)            # all actions (already being clipped) between -1 and 1
            if t%100 ==0 :
                 print('\rEpisode {}\t Timestamp: {}'.format(i_episode,t))
            env_info = env.step(actions)[brain_name]       # send all actions to tne environment
            next_states = env_info.vector_observations     # get next state (for each agent)
            rewards = env_info.rewards                     # get reward (for each agent)
            dones = env_info.local_done                    # see if episode finished

            for i_agent in range(num_agents):
                agent.step(states[i_agent],
                           actions[i_agent],
                           rewards[i_agent],
                           next_states[i_agent],
                           dones[i_agent], t)              # update the system

            score += rewards                               # update the score (for each agent)
            states = next_states                           # roll over states to next time step
            if np.any(dones):                              # exit loop if episode finished
                break
        scores_window.append(score)
        episode_scores.append(score)

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window) > 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode - 100,
                                                                                         np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
    return episode_scores

scores = train_ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 1	 Timestamp: 0
Episode 1	 Timestamp: 100
Episode 1	 Timestamp: 200
Episode 1	 Timestamp: 300
Episode 1	 Timestamp: 400
Episode 1	 Timestamp: 500
Episode 1	 Timestamp: 600
Episode 1	 Timestamp: 700
Episode 1	 Timestamp: 800
Episode 1	 Timestamp: 900
Episode 1	Average Score: 0.21
Episode 2	 Timestamp: 0
Episode 2	 Timestamp: 100
Episode 2	 Timestamp: 200
Episode 2	 Timestamp: 300
Episode 2	 Timestamp: 400
Episode 2	 Timestamp: 500
Episode 2	 Timestamp: 600
Episode 2	 Timestamp: 700
Episode 2	 Timestamp: 800
Episode 2	 Timestamp: 900
Episode 2	Average Score: 0.16
Episode 3	 Timestamp: 0
Episode 3	 Timestamp: 100
Episode 3	 Timestamp: 200
Episode 3	 Timestamp: 300
Episode 3	 Timestamp: 400
Episode 3	 Timestamp: 500
Episode 3	 Timestamp: 600
Episode 3	 Timestamp: 700
Episode 3	 Timestamp: 800
Episode 3	 Timestamp: 900
Episode 3	Average Score: 0.40
Episode 4	 Timestamp: 0
Episode 4	 Timestamp: 100
Episode 4	 Timestamp: 200
Episode 4	 Timestamp: 300
Episode 4	 Timestamp: 400
Episode 

Episode 28	 Timestamp: 900
Episode 28	Average Score: 1.40
Episode 29	 Timestamp: 0
Episode 29	 Timestamp: 100
Episode 29	 Timestamp: 200
Episode 29	 Timestamp: 300
Episode 29	 Timestamp: 400
Episode 29	 Timestamp: 500
Episode 29	 Timestamp: 600
Episode 29	 Timestamp: 700
Episode 29	 Timestamp: 800
Episode 29	 Timestamp: 900
Episode 29	Average Score: 1.40
Episode 30	 Timestamp: 0
Episode 30	 Timestamp: 100
Episode 30	 Timestamp: 200
Episode 30	 Timestamp: 300
Episode 30	 Timestamp: 400
Episode 30	 Timestamp: 500
Episode 30	 Timestamp: 600
Episode 30	 Timestamp: 700
Episode 30	 Timestamp: 800
Episode 30	 Timestamp: 900
Episode 30	Average Score: 1.41
Episode 31	 Timestamp: 0
Episode 31	 Timestamp: 100
Episode 31	 Timestamp: 200
Episode 31	 Timestamp: 300
Episode 31	 Timestamp: 400
Episode 31	 Timestamp: 500
Episode 31	 Timestamp: 600
Episode 31	 Timestamp: 700
Episode 31	 Timestamp: 800
Episode 31	 Timestamp: 900
Episode 31	Average Score: 1.42
Episode 32	 Timestamp: 0
Episode 32	 Timestam

Episode 56	 Timestamp: 300
Episode 56	 Timestamp: 400
Episode 56	 Timestamp: 500
Episode 56	 Timestamp: 600
Episode 56	 Timestamp: 700
Episode 56	 Timestamp: 800
Episode 56	 Timestamp: 900
Episode 56	Average Score: 1.42
Episode 57	 Timestamp: 0
Episode 57	 Timestamp: 100
Episode 57	 Timestamp: 200
Episode 57	 Timestamp: 300
Episode 57	 Timestamp: 400
Episode 57	 Timestamp: 500
Episode 57	 Timestamp: 600
Episode 57	 Timestamp: 700
Episode 57	 Timestamp: 800
Episode 57	 Timestamp: 900
Episode 57	Average Score: 1.42
Episode 58	 Timestamp: 0
Episode 58	 Timestamp: 100
Episode 58	 Timestamp: 200
Episode 58	 Timestamp: 300
Episode 58	 Timestamp: 400
Episode 58	 Timestamp: 500
Episode 58	 Timestamp: 600
Episode 58	 Timestamp: 700
Episode 58	 Timestamp: 800
Episode 58	 Timestamp: 900
Episode 58	Average Score: 1.42
Episode 59	 Timestamp: 0
Episode 59	 Timestamp: 100
Episode 59	 Timestamp: 200
Episode 59	 Timestamp: 300
Episode 59	 Timestamp: 400
Episode 59	 Timestamp: 500
Episode 59	 Timestamp:

KeyboardInterrupt: 

In [ ]:
env.close()